In [1]:
# demo from the ngsxfem library

from ngsolve import *
from xfem import *
from ngsolve.meshes import MakeStructured2DMesh

importing ngsxfem-2.1.2504


In [2]:
mesh = MakeStructured2DMesh(quads=False, nx=3, ny=2)
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [3]:
levelset = x - 0.8
gflset = GridFunction(H1(mesh))
InterpolateToP1(levelset, gflset)

In [4]:
DrawDC(gflset,-1,1,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [5]:
ci = CutInfo(mesh, gflset)
roots = ci.GetElementsOfType(NEG)
bads = ci.GetElementsOfType(IF)

In [6]:
#fes = L2(mesh, order=0)
fes = H1(mesh, order=1)

u, v = fes.TnT()
h = specialcf.mesh_size

# Creates an ElementAggregation based on a mesh, a list of root and a list of bad elements.
# It collects patches of elements that allow to stabilize bad cut elements by at least one good element (the root element).
EA = ElementAggregation(mesh, roots, bads)
# Computes and returns embedding matrix for a patchwise polynomial extension (realized through ghost penalties?)
# 本质上是一个稀疏矩阵 (sparse matrix) 对象，采用 CSR 存储格式
E = AggEmbedding(EA, fes)
# E = AggEmbedding(EA, fes, deformation=deformation) # if mesh deformation exists

In [7]:
# # element that is part of a (non-trivial) patch
Draw(BitArrayCF(EA.els_in_nontrivial_patch),mesh) # 被合并的那些单元
# # element that is not part of a (non-trivial) patch
Draw(BitArrayCF(EA.els_in_trivial_patch),mesh) # 未和其它单元合并的内部单元

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
# print(help(ElementAggregation))
# print(help(AggEmbedding))

In [9]:
gfu = GridFunction(fes)
# Draw(gfu) # all zero

In [ ]:
# print(help(E))

In [ ]:
index = 3
gfu_ori = GridFunction(fes)
gfu_ori.vec.data[index] = 1  # 此时表示第i个原始基函数
Draw(gfu_ori,mesh)

In [ ]:
# 创建一个与矩阵 E 列数相同维度的零向量 baser
# E.width 等于原始有限元空间fes的自由度数
baser = E.CreateRowVector() 
for i in range(1,E.width): # E.width is the number of columns of E
    baser[:] = 0
    baser[i] = 1
    # 此时表示第i个原始基函数
    gfu.vec.data = E * baser # 代表了第 i 个原始基函数通过聚合嵌入后的投影结果
    # Redraw()
    Draw(gfu,mesh,min=0,max=2)
    input(i)
print(" -- done -- ")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

1 


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…